In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [177]:
#loading data and dropping high cardinality object columns
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data = train_data.drop(['Cabin', 'Ticket', 'Name'], axis = 1)
test_data = test_data.drop(['Cabin', 'Ticket', 'Name'], axis = 1)
train_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [209]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

#processing the data

my_imputer = SimpleImputer(strategy='most_frequent')

cat_var = (test_data.dtypes == 'object')
object_cols = list(cat_var[cat_var].index)
object_cols.remove('Sex')

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

im_train_data = pd.DataFrame(my_imputer.fit_transform(train_data))
im_test_data = pd.DataFrame(my_imputer.fit_transform(test_data))

im_test_data.columns = test_data.columns
im_train_data.columns = train_data.columns

OH_cols_encoder_train = pd.DataFrame(OH_encoder.fit_transform(im_train_data[object_cols]))
OH_cols_encoder_test = pd.DataFrame(OH_encoder.fit_transform(im_test_data[object_cols]))

OH_cols_encoder_test.index = im_test_data.index
OH_cols_encoder_train.index = im_train_data.index

im_train_data = im_train_data.drop(object_cols, axis = 1)
im_test_data = im_test_data.drop(object_cols, axis = 1)

label_encoder = LabelEncoder()

im_train_data['Sex'] = label_encoder.fit_transform(im_train_data['Sex'])
im_test_data['Sex'] = label_encoder.fit_transform(im_test_data['Sex'])

final_train_data = pd.concat([im_train_data, OH_cols_encoder_train], axis = 1)
final_test_data = pd.concat([im_test_data, OH_cols_encoder_test], axis = 1)

nn = ['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare']

for n in nn:
    final_train_data[n] = pd.to_numeric(final_train_data[n])
    final_test_data[n] = pd.to_numeric(final_test_data[n])
print('OK')

final_train_data

OK


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,0,1,2
0,1,0,3,1,22.0,1,0,7.2500,0.0,0.0,1.0
1,2,1,1,0,38.0,1,0,71.2833,1.0,0.0,0.0
2,3,1,3,0,26.0,0,0,7.9250,0.0,0.0,1.0
3,4,1,1,0,35.0,1,0,53.1000,0.0,0.0,1.0
4,5,0,3,1,35.0,0,0,8.0500,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,0.0,0.0,1.0
887,888,1,1,0,19.0,0,0,30.0000,0.0,0.0,1.0
888,889,0,3,0,24.0,1,2,23.4500,0.0,0.0,1.0
889,890,1,1,1,26.0,0,0,30.0000,1.0,0.0,0.0


In [210]:
features = ['Pclass','Sex','Age','SibSp','Fare', 0, 1, 2]

X = final_train_data[features]
y = final_train_data.Survived

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, random_state = 1)

In [211]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = 50)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

pred = np.round(my_model.predict(X_valid)).astype(int)
mae = mean_absolute_error(pred, y_valid)
mae

[18:18:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.19553072625698323

In [193]:
final_model = XGBRegressor(n_estimator = 50)
final_model.fit(X,y,verbose=False)

prediction = np.round(final_model.predict(final_test_data[features])).astype(int)

filename = 'submission.csv'
pd.DataFrame({'PassengerId': final_test_data.PassengerId, 'Survived': prediction}).to_csv(filename, index=False)
prediction

[18:11:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,